<a href="https://colab.research.google.com/github/InesAgudoMartin/APRENDIZAJE_AUTOMATICO_23_24/blob/main/QUANTUM_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objective
We will put the class together in order to implement Grover with a really simple oracle.

- In class: 2 Qubit Grover
    1. From the |00> state to the |s> state
    2. Building the Phase Oracle and reflecting over |w perp>
    3. Building the Diffusion Operator and reflecting over |s>
- Homework: 3 Qubit Grover

### 0. From the |00> state to the |s> state
Now, this is not a follow along notebook, by this point, you should be able to create quantum circuits and change their states.

You now must create a 2 qubit, 2 bit quantum circuit, and create the perfect superposition state |s>.

In [ ]:
from qiskit import QuantumCircuit
#circuito con 2 qubits
qc=QuantumCircuit(2)
#aplicamos hadamard al primer qubit que hace una superposicion convirtiendo el estado 0 al 1
qc.h(0)
#aplicamos hadamard al siguente qubit
qc.h(1)
#barra para separar secciones del circuito
qc.barrier()
#lo mostramos
qc.draw('mpl')

1. Building the Phase Oracle and reflecting over |w perp>

Now our task is to create a Phase Oracle that will mark the |10> state. This means that it should change the phase of that state, and no other.

You can verify your statevector with the following piece of code:

---
```python
from qiskit import quantum_info
from qiskit.visualization import array_to_latex

initial_state = quantum_info.Statevector.from_instruction(qc) # <<< your QC there
array_to_latex(initial_state.data.reshape((-1,1)), prefix="|\\psi\\rangle =")
```
---


If you have applied the gates correctly, that means that you have already performed the reflection over |w perp>, congratulations.

In [ ]:
#puerta CZ: puerta de dos qubits que aplica una operación Z al segundo qubit (objetivo) solo si el primer qubit (control) está en el estado 1
#Si el qubit 0 está en el estado 1, entonces se aplica una puerta Z al qubit 1. De lo contrario, el qubit 1 permanece sin cambios.
qc.cz(0,1)
#aplicamos puerta Z al segundo qubit
#cambia la fase del qubit 1 si está en el estado 1, transformando 1> en -1> en. Si el qubit está en el estado 0 no cambia
qc.z(1)
qc.barrier()
qc.draw('mpl')

In [ ]:
#permite ver el estado cuantico qc en forma matematica

from qiskit import quantum_info
from qiskit.visualization import array_to_latex
#creamos vector de estado inicial
#creamos vector de estado a partir del circuito qc
initial_state = quantum_info.Statevector.from_instruction(qc) # <<< your QC there
#visualizar vector en formato latex
array_to_latex(initial_state.data.reshape((-1,1)), prefix="|\\psi\\rangle =")

# 2. Building the Diffusion Operator and reflecting over |s>

Now, the last thing we need to do in order to implement grover is to reflect our quantum state over the |s> state.

**We do that using the diffusion operator.**

In order to code this operator, as we are working in the Z Basis, we need to follow these steps:
 1. Change the basis of our qubits from the X basis (superposition basis) into the Z basis
 2. Apply a reflection over |0> (or |00> in our case), which is the equivalent of |s> in the Z basis
 3. Return our values to the X basis

Some tips:
- The hadamard gate changes from the X basis to the Z basis and viceversa.
- A reflection is an operation that changes the sign of every perpendicular vector to our axis, except our basis vector which is left as before

CONSTRUIR OPERADOR DE DIFUSION Y REFLEJAR SOBRE EL ESTADO S> PARA IMPLEMENTAR GROVER

In [ ]:
#aplicamos hadamard a los qubits para cambiar de base Z a base X
#Las puertas Hadamard transforman los qubits de la base de superposición (X) a la base computacional (Z)
qc.h(0)
qc.h(1)

#alicar reflexion sobre 00>
qc.cz(0,1) #puerta cz entre los dos qubits
#aplicamos puerta z a cada qubit cambiando su fase
qc.z(0)
qc.z(1)
#devolver nuestros valores a la base x: las puertas hadamard vuelven a transformar la base Z a la base X
qc.h(0)
qc.h(1)
#barrera: Etiqueta la barrera para indicar que esta sección representa el operador de difusión seguido del operador de fase
qc.barrier(label="U_s U_w|s>")
qc.draw('mpl')

Running our circuit

In [ ]:
# Simulate your circuit to see if the circuit works
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session

#qc.measure(0,0)
#qc.measure(1, 1)
qc.measure_all() #mide todos los qubits del circuito
service = QiskitRuntimeService() #inicializa el servicio
backend_simulator = service.get_backend('ibmq_qasm_simulator')# Get your backend here, simulador

#### YOUR CODE TO LAUNCH THE CIRCUIT HERE
#ejecuatar el circuito en el simulador
with Session(service=service, backend=backend_simulator) as session:
    sampler = Sampler(session=session)
    job = sampler.run(circuits=qc)
    print(job.result())
from qiskit.visualization import plot_histogram
plot_histogram(job.result().quasi_dists[0])
#muestra circuito e histograma
display(qc.draw('mpl'),plot_histogram(job.result().quasi_dists[0]))

#RESUMEN CÓDIGO Y PASOS REALIZADOS
Objetivo
Implementar el algoritmo de Grover utilizando un oráculo simple en Qiskit, comenzando con un circuito de 2 qubits y luego extendiéndolo a 3 qubits.

####Pasos Generales Realizados:

###Creación del Circuito Cuántico con 2 Qubits:

Se crea un circuito cuántico con 2 qubits y se aplica la puerta Hadamard a cada qubit para ponerlos en un estado de superposición perfecta ∣𝑠>

###Construcción del Oráculo de Fase y Reflexión sobre ∣𝑤⊥>

Se implementa un oráculo que marca el estado ∣10> aplicando una puerta CZ seguida de una puerta Z al segundo qubit.

Se verifica el vector de estado resultante para asegurar que la operación se ha realizado correctamente.

###Construcción del Operador de Difusión y Reflexión sobre ∣𝑠>:

Se cambia la base de los qubits de X a Z aplicando puertas Hadamard.

Se aplica una reflexión sobre el estado ∣00> usando puertas CZ y Z.

Se devuelve la base de los qubits a X aplicando nuevamente puertas Hadamard.

Se añade una barrera para visualizar la separación de secciones en el circuito.


###Simulación del Circuito:
Se añaden mediciones a todos los qubits del circuito.

Se inicializa el servicio de Qiskit Runtime y se selecciona el backend del simulador.

Se ejecuta el circuito en el simulador y se obtienen los resultados.

Se visualiza el circuito y los resultados de la simulación en forma de histograma.